In [ ]:
%pip install torchcrepe

In [ ]:
%pip install pretty_midi

In [ ]:
import torchcrepe
import pretty_midi
import matplotlib.pyplot as plt
import numpy as np
import torch

# 오디오 파일 로드
audio, sr = torchcrepe.load.audio('/content/file.wav')

# 스테레오 오디오일 경우 모노로 변환
if audio.ndim == 2 and audio.shape[0] == 2:
    audio = audio.mean(dim=0, keepdim=True)

# audio 모양을 [1, samples]로 맞춤
if audio.ndim == 1:
    audio = audio.unsqueeze(0)

# 피치 추출에 필요한 파라미터 설정
hop_length = int(sr / 200.)
fmin = 50 # G#1 ~
fmax = 1000 # ~B5
model = 'full'
device = 'cuda:0'
batch_size = 2048

# 오디오 신호의 음량(Amplitude) 계산
amplitude = audio.abs()

# 프레임별 음량 계산 (hop_length에 맞춰 평균 음량 구하기)
frame_amplitude = torch.nn.functional.avg_pool1d(amplitude, hop_length, hop_length)

# 음량이 너무 낮은 경우 (무음으로 간주할 임계값 설정)
silence_threshold = 0.01  # 음량이 0.01보다 작은 경우 무음 처리

# 피치 추출
pitch = torchcrepe.predict(
    audio,
    sr,
    hop_length,
    fmin,
    fmax,
    model,
    batch_size=batch_size,
    device=device
)

# 두 배열의 길이를 일치시키기 위해 짧은 쪽에 맞춤
min_length = min(pitch.shape[-1], frame_amplitude.shape[-1])
pitch = pitch[..., :min_length].squeeze().cpu().numpy()
frame_amplitude = frame_amplitude[..., :min_length].squeeze().cpu().numpy()

# 음량이 작은 구간을 무음으로 처리
pitch[frame_amplitude < silence_threshold] = 0

# 시간 축 생성 (프레임당 시간)
time_per_frame = hop_length / sr
time = np.arange(pitch.shape[0]) * time_per_frame

# 피치가 0이 아닌 유효한 피치 값만 추출
valid_pitch = pitch[pitch > 0]

# 최고음과 최저음 계산
if valid_pitch.size > 0:
    max_pitch_hz = np.max(valid_pitch)
    min_pitch_hz = np.min(valid_pitch)

    # Hz를 MIDI 노트로 변환
    max_note_number = pretty_midi.hz_to_note_number(max_pitch_hz)
    min_note_number = pretty_midi.hz_to_note_number(min_pitch_hz)

    # MIDI 노트 번호를 음계로 변환
    max_note_name = pretty_midi.note_number_to_name(int(max_note_number))
    min_note_name = pretty_midi.note_number_to_name(int(min_note_number))

    # 출력
    print(f"최고음: {max_note_name}, 최저음: {min_note_name}")
else:
    print("유효한 피치 정보가 없습니다.")

# === 피치 그래프 그리기 ===
plt.figure(figsize=(10, 6))
plt.plot(time, pitch, label='Pitch (Hz)', color='b')
plt.xlabel('Time (s)')
plt.ylabel('Pitch (Hz)')
plt.title('Pitch Contour with Silence Removal')
plt.grid(True)
plt.legend()
plt.show()

# === MIDI 파일 생성 ===
midi = pretty_midi.PrettyMIDI()
instrument = pretty_midi.Instrument(program=0)  # 기본 피아노 악기 설정

# 피치 정보를 MIDI로 변환
for i, f in enumerate(pitch):
    if f > 0:  # 무음이 아닌 경우만 처리
        note_number = int(pretty_midi.hz_to_note_number(f))  # 주파수를 MIDI 노트 번호로 변환
        start_time = i * time_per_frame
        end_time = start_time + time_per_frame
        note = pretty_midi.Note(velocity=100, pitch=note_number, start=start_time, end=end_time)
        instrument.notes.append(note)

# 악기 추가 및 MIDI 파일 저장
midi.instruments.append(instrument)
midi.write('/content/output.mid')

print("MIDI 파일 생성 완료: output.mid")